In [ ]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, DepthwiseConv2D, Activation, AveragePooling2D, Dropout, SeparableConv2D, Flatten, Dense, SpatialDropout2D
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Model
from tensorflow.keras.utils.vis_utils import plot_model

class EEGNet:
    def __init__(self, numClasses, chans=124, samples=1059, 
                 dropoutRate=0.5, kernLength=64, F1=8, 
                 D=2, F2=16, normRate=0.25, dropoutType='Dropout'):
        print('*************Building EEGNet Based Architecture*****************')
        self.nbClasses = numClasses
        self.chans = chans
        self.samples = samples
        self.dropoutRate = dropoutRate
        self.kernLength = kernLength
        self.F1 = F1
        self.D = D
        self.F2 = F2
        self.normRate = normRate
        self.dropoutType = dropoutType

        if self.dropoutType == 'SpatialDropout2D':
            self.dropoutType = SpatialDropout2D
        elif self.dropoutType == 'Dropout':
            self.dropoutType = Dropout
        else:
            raise ValueError('dropoutType must be one of SpatialDropout2D or Dropout, passed as a string.')

    def buildModel(self):
        input1 = Input(shape=(self.chans, self.samples, 1))

        # Block 1
        block1 = Conv2D(self.F1, (1, self.kernLength), padding='same', use_bias=False)(input1)
        block1 = Conv2D(self.F1, (1, self.kernLength), padding='same', use_bias=False)(block1)
        block1 = BatchNormalization()(block1)
        block1 = DepthwiseConv2D((self.chans, 1), use_bias=False, depth_multiplier=self.D, depthwise_constraint=max_norm(1.))(block1)
        block1 = BatchNormalization()(block1)
        block1 = Activation('elu')(block1)
        block1 = AveragePooling2D((1, 4))(block1)
        block1 = self.dropoutType(self.dropoutRate)(block1)

        # Block 2
        block2 = SeparableConv2D(self.F2, (1, 16), use_bias=False, padding='same')(block1)
        block2 = BatchNormalization()(block2)
        block2 = Activation('elu')(block2)
        block2 = AveragePooling2D((1, 8))(block2)
        block2 = self.dropoutType(self.dropoutRate)(block2)

        # Flatten and Dense layers
        flatten = Flatten(name='flatten')(block2)
        dense = Dense(self.nbClasses, name='dense', kernel_constraint=max_norm(self.normRate))(flatten)
        softmax = Activation('softmax', name='softmax')(dense)

        return Model(inputs=input1, outputs=softmax)

# Instantiate the model
eeg_net = EEGNet(numClasses=10)
model = eeg_net.buildModel()


